In [7]:
import os
import glob
import random
from pydub import AudioSegment
import pandas as pd
augmented = []
original_df = pd.read_csv('Train.csv')
noise_files = glob.glob('musan_noise/*/*.wav')
random.seed(2023)


def speed_change(sound, speed=1.0):
    sound_with_altered_frame_rate = sound._spawn(sound.raw_data, overrides={"frame_rate": int(sound.frame_rate * speed)})
    return sound_with_altered_frame_rate.set_frame_rate(sound.frame_rate)


def augmentation(fname, speeds):
    os.makedirs('audio_augmented', exist_ok=True)
    audio = AudioSegment.from_file(fname)
    basename = os.path.basename(fname).split('.')[0]
    for speed in speeds:
        audio = speed_change(audio, speed)
        noise = AudioSegment.from_file(random.choice(noise_files))
        noise = noise.apply_gain(-random.randint(5, 10))
        audio = audio.overlay(noise, times=max(1, int(len(audio) / len(noise) * 0.5)))  # overlay for half of the audio
        audio.export(f'audio_augmented/{basename}_{speed}.wav', format='wav')
        augmented.append({'path': f'audio_augmented/{basename}_{speed}.wav', 'annotation': original_df[original_df['path'] == fname]['annotation'].values[0]})

In [8]:
for fname in glob.glob('audio/train*.wav'):
    fname = fname.replace('\\', '/')
    augmentation(f'{fname}', [0.9, 1, 1.1, 1.2])

In [10]:
augmented_df = pd.DataFrame(augmented)
augmented_df.to_csv('Train_augmented.csv', index=False)